# Combining Datasets: [Merge and Join](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)
______________________________

* Combining datasets based on Relational Algebra:
    * a formal set of rules for manipulating relational data
    * the conceptual foundation of operations available in most databases
    * it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset
* Pandas implements several of fundamental building-blocks in:
     * ``pd.merge()`` function 
     * ``join()`` method of ``Series`` and ``Dataframe``

## 1. ``pd.merge()``  --  main interface for high-performance, **in-memory** join and merge operations 
________________________________________________________________

#### 1.1. Categories of joins by ``pd.merge()``
_____________________
* ``pd.merge()`` implements a number of types of joins: 
   * one-to-one
   * many-to-one
   * many-to-many
* the type of join depends on the form of the input data
* ``pd.merge()`` combines information into a single ``DataFrame``

In [1]:
import pandas as pd
import numpy as np

In [2]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [3]:
pd.merge?

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'str' = 'inner',
    on: 'IndexLabel | None' = None,
    left_on: 'IndexLabel | None' = None,
    right_on: 'IndexLabel | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool' = True,
    indicator: 'bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

.. warning::

    If both key columns contain rows where the key is a null

#### 1.2. One-to-one joins
___________________
* in many ways is very similar to the column-wise concatenation 
* default behavior of ``pd.merge()``: it looks for one or more **matching column names** between the two inputs, and uses this as the **key**
* result of the merge is a new ``DataFrame`` 
* order of entries in each column is not necessarily maintained 
* ``pd.merge()`` in general discards the index, except in the special case of merges by index 

In [4]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [5]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [6]:
df3a = pd.merge(df2, df1)
df3a

,employee,hire_date,group
0,Lisa,2004,Engineering
1,Bob,2008,Accounting
2,Jake,2012,Engineering
3,Sue,2014,HR


* ``pd.merge()`` recognizes that each ``DataFrame`` has an ``"employee"`` column, and automatically joins using this column as a key
* order of the "employee" column differs between ``df1`` and ``df2``, and ``pd.merge()`` correctly accounts for this

#### 1.3. Many-to-one joins
___________________

* one of the two key columns contains duplicate entries
* the resulting ``DataFrame`` will preserve those duplicate entries as appropriate

In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

In [8]:
display('df4', 'df3', 'pd.merge(df4, df3)')

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

pd.merge(df4, df3)
         group supervisor employee  hire_date
0   Accounting      Carly      Bob       2008
1  Engineering      Guido     Jake       2012
2  Engineering      Guido     Lisa       2004
3           HR      Steve      Sue       2014

* The resulting ``DataFrame`` has an aditional column with the ``"supervisor"`` information, where the information is repeated in one or more locations as required by the inputs.

#### 1.4. Many-to-many joins
____________________

* If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.

 ``df5``: there are one or more skills associated with a particular group; by performing a many-to-many join, the skills associated with any individual person will be recovered:

In [9]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


#### 1.5. Specification of the Merge Key if the column names will not match so nicely
_______________

* **``on``** --  takes a column name or a list of column names (if both the left and right ``DataFrame`` have the specified column name) 
_____________

In [12]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

* **``left_on``** and **``right_on``** -- to merge two datasets with different column names
____________________________________

In [13]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [14]:
display('df1', 'df3', "pd.merge(df1, df3, left_on='employee', right_on='name')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on='employee', right_on='name')
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

* The result has a redundant column that can be dropped if desired by using the ``DataFrame.drop()``
______________________

In [ ]:
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name").drop("name", axis=1)')

* **``left_index``** and/or **``right_index``** -- the using for  the key specification
_________________________________

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

In [ ]:
display('df1a', 'df2a', "pd.merge(df1a, df2a)")# MergeError: No common columns to perform merge on

* Combination -- mixing indices and columns ``left_index`` with ``right_on`` or ``left_on`` with ``right_index`` 
____________________________________

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

## 2. ``DataFrame.join()`` 
_____________________________________

* Join columns with other `DataFrame` either on index or on a key column
* Efficiently join multiple `DataFrame` objects by index at once by passing a list


In [ ]:
pd.DataFrame.join?

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

## 3. Specifying Arithmetic for joins
____________________________

#### 3.1. ``how="inner"`` -- **inner join** by default -- the result contains the *intersection* of the two sets of inputs

In [15]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])

In [16]:
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

In [ ]:
pd.merge(df6, df7, how='inner')

#### 3.2. Other options for ``how``
_________________

* ``how='outer'`` --   returns a join over the union of the input columns, and fills in all missing values with NaN

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

* ``how='left'`` -- return joins over the left entries 

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

* ``how='right'`` -- return joins over the right entries 

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

## 4. Overlapping column names:  ``suffixes`` keyword
_____________________________________

A case where two input ``DataFrame`` have conflicting column names

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})

In [18]:
display('df8', 'df9')

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4
,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


* Because the output would have two conflicting column names, the ``merge()``  automatically appends a suffix ``_x`` or ``_y`` to make the output columns unique:

In [19]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

* using  to make the output columns unique
* ``suffixes`` work in any of the possible join patterns, and work also if there are multiple overlapping columns

In [ ]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')